# Visualize Point Toggle Bg

Notebook to plot selected channels for 1 point, a.k.a. FoV.
It also plots the spectrum in TOF bin units and in mass units.
This version of the script takes 2 images, plots them in the same axis object and allows the user to toggle between both of them: once the notebook has run, select the desired plot and press 't' to toggle the visibility of both plots. This is useful for instance, in order to visualize the change in a channel before and afer slide bg removal.

Accepted format for the images: IONpath's **MIBItiff** as from the MIBI/O software or the MIBItracker.

Please update the parameters in the **input parameters** section before running the notebook.

# load headers

In [ ]:
# magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2

#imports
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mibidata import mibi_image as mi, tiff
import visualize_data as viz

# input parameters

In [ ]:
data_path = '../sample_data'

slide_num = 1
point_num = 1
run_name = 'CohortSlide1'

# load tiff
tiff_file = os.path.join(data_path,
                         f'{run_name}/{run_name}_TIFF',
                         'bg_au_050_ta_020',
                         f'Point{point_num}_RowNumber0_Depth_Profile0.tiff')
# load bg tiff
tiff_file_bg = os.path.join(data_path,
                            f'{run_name}/{run_name}_TIFF',
                            f'bg_none',
                            f'Point{point_num}_RowNumber0_Depth_Profile0.tiff')

# load spectrum csv file
spectrum_file = os.path.join(data_path,
                             f'{run_name}/Point{point_num}/RowNumber0/Depth_Profile0/',
                             'AccumulatedSpectrum.csv')
# load panel
panel_file = os.path.join(data_path,
                          'Panel.csv')

# select channels
a_masses = np.array([89, 113, 115, 146, 148, 149, 159, 181, 197])
a_masses.sort()
print(f'a_masses = {a_masses}')

#anonymize_targets = True
anonymize_targets = False

# read panel

In [ ]:
panel_df = viz.read_panel_from_csv(panel_file, anonymize_targets=anonymize_targets)
#display(panel_df)

# transform masses to targets for selected channels
l_targets = []
for mass in a_masses:
    l_targets.append(panel_df['Target'][panel_df['Mass'] == mass].values[0])
print(f'l_targets = {l_targets}')

# plot spectrum

In [ ]:
# read spectrum csv file
spectrum_df = viz.read_spectrum_from_csv(spectrum_file)
#display(spectrum_df)

# light plot style
plt.style.use('default')

spec_axes = []

# plot TOF bin spectrum
spec_axes.append(viz.plot_spectrum(spectrum_df, plot_mass=False, plot_wide=True,
                                   ax=None,
                                   title='TOF bin spectrum'))

# plot mass spectrum
spec_axes.append(viz.plot_spectrum(spectrum_df, plot_mass=True, plot_wide=True,
                                   ax=None,
                                   title='mass spectrum'))

# plot vertical lines for know masses
df_known_masses = viz.get_know_masses_df()
for _, row in df_known_masses.iterrows():
    #print(f'Element {row["Element"]}, mass {row["Mass"]}')
    spec_axes[1].axvline(x=row['Mass'], color='gray', linestyle=':', linewidth=1)
#display(df_known_masses)

# plot vertical lines for selected channels
for mass in a_masses:
    spec_axes[1].axvline(x=mass, color='black', linestyle='--', linewidth=1)

# plot images

In [ ]:
# read MIBItiff image file
image = tiff.read(tiff_file)
if not anonymize_targets:
    #print(f'metadata {image.metadata}')
    #print(f'channels {image.channels}') # sorted by target name
    print(f'channels {sorted(image.channels)}') # sorted by mass

# read MIBItiff bg image file
image_bg = tiff.read(tiff_file_bg)

# dark plot style
plt.style.use('dark_background')

# loop over channels
im_axes = []
for ch_order, channel in enumerate(sorted(image.channels)):
    mass = channel[0]
    target = channel[1]
    #print() # debug
    #print(f'channel {ch_order}: {channel}, mass {mass}, target {target}') # debug
    #if ch_order < 5 or mass == 146: # debug
    #if mass > 85: # low masses are not interesting
    #if mass > 85 and mass < 180 and mass != 128: # all biological channels
    #if mass > 0: # all masses
    #if target in l_targets: # selected channels
    if mass in a_masses: # selected channels (anonymous version)
        print()
        if anonymize_targets:
            try:
                target = l_targets[np.where(a_masses == mass)[0][0]]
            except:
                target = 'dummy'
            finally:
                print('WARNING! TIFF files and \'channel\' variable might not be anonymized!')
#         else:
#             print(f'channel {ch_order}: {channel}, mass {mass}, target {target}') # debug (channel var not anonymized!)
        print(f'channel {ch_order}: mass {mass}, target {target}')
        im = image[mass]
        im_bg = image_bg[mass]
        counts = im.sum()
        im_axes.append(viz.plot_toggle_image(im, im_bg,
                                             ax=None,
                                             #title=str(channel) + ' ' + str("{:.2e}".format(counts)), # debug (not anonymized)
                                             title=f'({mass}, \'{target}\')' + ' ' + str("{:.2e}".format(counts)),
                                             brighten_image=True, hi_res=True))
                                             #brighten_image=True, hi_res=False))
        #break # debug (show only 1 plot)